In [1]:
# pip install yfinance --upgrade --no-cache-dir

  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23914 sha256=73ff8778f1289f676caad856e5cc91d0a75a679d366dd1df4588e70561299a06
  Stored in directory: C:\Users\Maksim\AppData\Local\Temp\pip-ephem-wheel-cache-6snr5mt4\wheels\fe\87\8b\7ec24486e001d3926537f5f7801f57a74d181be25b11157983
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8374 sha256=f5553a344a67eb26b0b66b2559b1e23a131447f7ec17480e836f56c1a234999b
  Stored in directory: C:\Users\Maksim\AppData\Local\Temp\pip-ephem-wheel-cache-6snr5mt4\wheels\ae\25\47\4d68431a7ec1b6c4b5233365934b74c1d4e665bf5f968d363a
Successfully built yfinance multitasking
  Attempting uninstall: lxml
    Found existing installation: lxml 4.5.0
    Uninstalling lxml-4.5.0:
      Successfully uninstalled lxml-4.5.0
Note: you may need to restart the kernel to use updated packages.


In [42]:
# pip install pandas_datareader

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Knowledge Discovery in Databases

In [1]:
import yfinance as yf

In [44]:
import datetime
from dateutil.relativedelta import relativedelta

In [3]:
import requests

In [73]:
import time

In [43]:
import pandas as pd
import pandas_datareader.data as web

In [40]:
# pd.core.common.is_list_like = pd.api.types.is_list_like

In [52]:
# table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
# df = table[0]
# df.to_csv('S&P500-Info.csv')
# df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])

In [6]:
df_sp = pd.read_csv('S&P500-Info.csv')
list_of_tickers = df_sp.Symbol

In [67]:
df['GICS Sector'].unique()

array(['Industrials', 'Health Care', 'Information Technology',
       'Communication Services', 'Consumer Discretionary', 'Utilities',
       'Financials', 'Materials', 'Real Estate', 'Consumer Staples',
       'Energy'], dtype=object)

In [45]:
today = datetime.datetime.now().date()
oldest_day = today - relativedelta(years=2)

In [47]:
SP500 = web.DataReader(['sp500'], 'fred', oldest_day, today)

In [48]:
SP500

,sp500
DATE,
2019-10-11,2970.27
2019-10-14,2966.15
2019-10-15,2995.68
2019-10-16,2989.69
2019-10-17,2997.95
...,...
2021-10-01,4357.04
2021-10-04,4300.46
2021-10-05,4345.72


In [81]:
total_df = pd.DataFrame()
bad_count = 0
for ticker in list_of_tickers:
    # request data
    company = yf.Ticker(ticker)
    df_quarterly_financials = company.quarterly_financials
    df_quarterly_balance_sheet = company.quarterly_balance_sheet
    df_quarterly_cashflow = company.quarterly_cashflow
    df_earnings = company.quarterly_earnings
    hist = company.history(period="2y")
    
    # time
    start = df_quarterly_cashflow.columns[3]
    future_date = start + datetime.timedelta(days=252)
    finish = start + datetime.timedelta(days=252) if future_date <= today else today
    
    # variables
    try:
        total_liab = df_quarterly_balance_sheet.loc['Total Liab'][start]
        total_equity = df_quarterly_balance_sheet.loc['Total Stockholder Equity'][start]
        total_assets = df_quarterly_balance_sheet.loc['Total Assets'][start]
        current_assets = df_quarterly_balance_sheet.loc['Total Current Assets'][start]
        cash = df_quarterly_balance_sheet.loc['Cash'][start]
        net_income = df_quarterly_cashflow.loc['Net Income'][start]
        earnings = df_earnings['Earnings'][0]
        number_of_stocks = df_quarterly_balance_sheet.loc['Common Stock'][start]
        start_close = hist.loc[start]['Close']
        finish_close = hist.loc[finish]['Close']
        operating_cashflow = df_quarterly_cashflow.loc['Total Cash From Operating Activities'][start]
        interest_expense = df_quarterly_financials.loc['Interest Expense'][start]
        ebit = df_quarterly_financials.loc['Ebit'][start]
        total_revenue = df_quarterly_financials.loc['Total Revenue'][start]
    except KeyError:
        bad_count += 1
        print(f'Bad company {ticker}. Overall number {bad_count}')
        continue
    
    # ratios 
    try:
        # https://www.investopedia.com/terms/d/debtequityratio.asp
        debt_to_equity = total_liab / total_equity

        # https://www.investopedia.com/terms/c/currentratio.asp
        current_ratio =  current_assets / total_liab

        # https://www.investopedia.com/terms/c/cash-ratio.asp
        cash_ratio = cash / total_liab

        # https://www.investopedia.com/terms/r/returnonassets.asp
        return_on_assets = net_income / total_assets

        # https://www.investopedia.com/terms/e/eps.asp
        eps = earnings / number_of_stocks
    
        # https://www.investopedia.com/terms/p/price-to-salesratio.asp
        price_to_sales = start_close / finish_close

        # https://www.investopedia.com/terms/p/price-to-cash-flowratio.asp
        price_to_cashflow = start_close / (operating_cashflow / number_of_stocks)

        # https://www.investopedia.com/terms/s/shareholderequityratio.asp
        shareholder_equity = total_equity / total_assets
    
        # https://www.investopedia.com/terms/i/interestcoverageratio.asp
        interest_coverage = ebit / interest_expense

        # https://www.investopedia.com/terms/n/net_margin.asp
        net_profit_margin = net_income / total_revenue
        
    except TypeError:
        bad_count += 1
        print(f'Bad company {ticker}. Overall number {bad_count}')
        continue
        
    # targets
    regression_target = start_close / finish_close  # prrice_change, regression
    target = (regression_target < 1)  # overweight, for classification
    SP_change = SP500.loc[start]['sp500'] / SP500.loc[finish]['sp500']
    SP_target = SP_change > regression_target  # overweight, for classification
    
    df_company = pd.DataFrame({'debt_to_equity': debt_to_equity, 'current_ratio': current_ratio, 'cash_ratio': cash_ratio,
                          'return_on_assets': return_on_assets, 'eps': eps, 'price_to_sales': price_to_sales,
                          'price_to_casflow': price_to_cashflow, 'shareholder_equity': shareholder_equity, 
                          'interest_coverage': interest_coverage, 'net_profit_margin': net_profit_margin,
                          'regression_target': regression_target, 'target': target, 'SP_target': SP_target}, index=[ticker])
    
    
    total_df = pd.concat([total_df, df_company])
    print(ticker, len(total_df))
    time.sleep(1)
    

MMM 1
ABT 2
ABBV 3
Bad company ABMD. Overall number 1
ACN 4
Bad company ATVI. Overall number 2
Bad company ADBE. Overall number 3
Bad company AMD. Overall number 4
Bad company AAP. Overall number 5
AES 5
AFL 6
Bad company A. Overall number 6
APD 7
AKAM 8
ALK 9
ALB 10
ARE 11
Bad company ALGN. Overall number 7
ALLE 12
LNT 13
ALL 14
GOOGL 15
GOOG 16
MO 17
AMZN 18
AMCR 19
AEE 20
AAL 21
AEP 22
Bad company AXP. Overall number 8
AIG 23
AMT 24
AWK 25
AMP 26
ABC 27
AME 28
AMGN 29
APH 30
Bad company ADI. Overall number 9
ANSS 31
ANTM 32
AON 33
AOS 34
APA 35
Bad company AAPL. Overall number 10
Bad company AMAT. Overall number 11
APTV 36
ADM 37
Bad company ANET. Overall number 12
AJG 38
AIZ 39
T 40
ATO 41
Bad company ADSK. Overall number 13
ADP 42
Bad company AZO. Overall number 14
AVB 43
Bad company AVY. Overall number 15
Bad company BKR. Overall number 16
BLL 44
Bad company BAC. Overall number 17
Bad company BBWI. Overall number 18
BAX 45
BDX 46
- BRK.B: No data found, symbol may be delisted


TypeError: can only concatenate str (not "datetime.timedelta") to str

In [90]:
for ticker in list_of_tickers[65:]:
    # request data
    company = yf.Ticker(ticker)
    df_quarterly_financials = company.quarterly_financials
    df_quarterly_balance_sheet = company.quarterly_balance_sheet
    df_quarterly_cashflow = company.quarterly_cashflow
    df_earnings = company.quarterly_earnings
    hist = company.history(period="2y")
    if len(df_quarterly_financials) == 0:
        bad_count += 1
        print(f'No data for company {ticker}. Overall number {bad_count}')
        continue
        
    
    # time
    start = df_quarterly_cashflow.columns[3]
    future_date = start + datetime.timedelta(days=252)
    finish = start + datetime.timedelta(days=252) if future_date <= today else today
    
    # variables
    try:
        total_liab = df_quarterly_balance_sheet.loc['Total Liab'][start]
        total_equity = df_quarterly_balance_sheet.loc['Total Stockholder Equity'][start]
        total_assets = df_quarterly_balance_sheet.loc['Total Assets'][start]
        current_assets = df_quarterly_balance_sheet.loc['Total Current Assets'][start]
        cash = df_quarterly_balance_sheet.loc['Cash'][start]
        net_income = df_quarterly_cashflow.loc['Net Income'][start]
        earnings = df_earnings['Earnings'][0]
        number_of_stocks = df_quarterly_balance_sheet.loc['Common Stock'][start]
        start_close = hist.loc[start]['Close']
        finish_close = hist.loc[finish]['Close']
        operating_cashflow = df_quarterly_cashflow.loc['Total Cash From Operating Activities'][start]
        interest_expense = df_quarterly_financials.loc['Interest Expense'][start]
        ebit = df_quarterly_financials.loc['Ebit'][start]
        total_revenue = df_quarterly_financials.loc['Total Revenue'][start]
    except KeyError:
        bad_count += 1
        print(f'No field for company {ticker}. Overall number {bad_count}')
        continue
    
    # ratios 
    try:
        # https://www.investopedia.com/terms/d/debtequityratio.asp
        debt_to_equity = total_liab / total_equity

        # https://www.investopedia.com/terms/c/currentratio.asp
        current_ratio =  current_assets / total_liab

        # https://www.investopedia.com/terms/c/cash-ratio.asp
        cash_ratio = cash / total_liab

        # https://www.investopedia.com/terms/r/returnonassets.asp
        return_on_assets = net_income / total_assets

        # https://www.investopedia.com/terms/e/eps.asp
        eps = earnings / number_of_stocks
    
        # https://www.investopedia.com/terms/p/price-to-salesratio.asp
        price_to_sales = start_close / finish_close

        # https://www.investopedia.com/terms/p/price-to-cash-flowratio.asp
        price_to_cashflow = start_close / (operating_cashflow / number_of_stocks)

        # https://www.investopedia.com/terms/s/shareholderequityratio.asp
        shareholder_equity = total_equity / total_assets
    
        # https://www.investopedia.com/terms/i/interestcoverageratio.asp
        interest_coverage = ebit / interest_expense

        # https://www.investopedia.com/terms/n/net_margin.asp
        net_profit_margin = net_income / total_revenue
        
    except TypeError:
        bad_count += 1
        print(f'Some zero company {ticker}. Overall number {bad_count}')
        continue
        
    # targets
    regression_target = start_close / finish_close  # prrice_change, regression
    target = (regression_target < 1)  # overweight, for classification
    SP_change = SP500.loc[start]['sp500'] / SP500.loc[finish]['sp500']
    SP_target = SP_change > regression_target  # overweight, for classification
    
    df_company = pd.DataFrame({'debt_to_equity': debt_to_equity, 'current_ratio': current_ratio, 'cash_ratio': cash_ratio,
                          'return_on_assets': return_on_assets, 'eps': eps, 'price_to_sales': price_to_sales,
                          'price_to_casflow': price_to_cashflow, 'shareholder_equity': shareholder_equity, 
                          'interest_coverage': interest_coverage, 'net_profit_margin': net_profit_margin,
                          'regression_target': regression_target, 'target': target, 'SP_target': SP_target}, index=[ticker])
    
    
    total_df = pd.concat([total_df, df_company])
    print(ticker, len(total_df))
    time.sleep(1)

No field for company BBY. Overall number 19
BIO 47
TECH 48
BIIB 49
BLK 50
Some zero company BK. Overall number 20
BA 51
No field for company BKNG. Overall number 21
BWA 52
BXP 53
BSX 54
BMY 55
No field for company AVGO. Overall number 22
BR 56
BRO 57
- BF.B: None
No data for company BF.B. Overall number 23
CHRW 58
No field for company CDNS. Overall number 24
CZR 59
No field for company CPB. Overall number 25
Some zero company COF. Overall number 26
CAH 60
KMX 61
CCL 62
CARR 63
CTLT 64
CAT 65
CBOE 66
CBRE 67
CDW 68
No field for company CE. Overall number 27
CNC 69
CNP 70
CDAY 71
CERN 72
CF 73
No field for company CRL. Overall number 28
Some zero company SCHW. Overall number 29
No field for company CHTR. Overall number 30
CVX 74
CMG 75
CB 76
CHD 77
CI 78
CINF 79
CTAS 80
No field for company CSCO. Overall number 31
Some zero company C. Overall number 32
Some zero company CFG. Overall number 33
CTXS 81
CLX 82
CME 83
CMS 84
KO 85
CTSH 86
CL 87
CMCSA 88
Some zero company CMA. Overall number 

In [95]:
total_df.to_csv('financial_data.csv')